In [2]:
# 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

# 데이터 불러오기
import pandas as pd
news = pd.read_csv("/content/drive/My Drive/DSCapstone/Newsdata_URL_TEXT.csv", index_col=0)

news.head()

Mounted at /content/drive


URL  \
0  https://www.hani.co.kr/arti/society/rights/111...   
1  https://www.joongboo.com/news/articleView.html...   
2  https://imnews.imbc.com/news/2024/society/arti...   
3              https://www.news1.kr/articles/5152573   
4  https://www.chosun.com/national/transport-envi...   

                                       TITLE                 DATE  \
0           서울 1~8호선 지하철, 오늘 오전 9시부터 ‘경고 파업’  2023-11-08 22:20:00   
1                  폭염 속 지하철 1호선 지연… 시민들 ‘짜증’  2023-08-01 00:00:00   
2          강설로 서울 지하철 운행 차질..2·5·7호선 전 구간 지연  2024-02-22 07:44:00   
3  "월요일부터 지각" 지하철 1호선 지연 운행에 시민 불편…"노조 준법투쟁"  2023-08-28 08:26:00   
4                 1호선 또...지연 운행으로 퇴근길 시민들 불편  2024-03-18 00:00:00   

                                             ARTICLE  
0   서울지하철 1~8호선을 운영하는 서울교통공사(공사) 노동조합이 9일 아침 9시부터...  
1  전국 대부분 지역에 폭염경보가 내려진 1일 지하철 1호선이 연착돼 이용객들이 불편을...  
2  밤사이 내려 얼어붙은 눈으로 서울 지하철 2호선, 5호선, 7호선의 전구간 운행이 ...  
3   철도노조가 준법투쟁에 돌입한 24일 서울역 대합실에 철도노조 태업 관련 안내문이 ...  
4   18일 오후 경기도 의정부시 수도권 전철 1호선 회룡역에서 서울 1호선 도봉산역 ...

In [41]:
#!pip install langchain_openai

In [4]:
#!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.4 MB/s eta 0:00:00


In [5]:
import getpass
import os
# 환경변수에 OpenAI API 키 저장 (사용자 입력으로 안전하게)
os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [6]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4-turbo") # GPT-4 모델로 챗봇 초기화

from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory,ConversationSummaryMemory

from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

#템플릿 생성 - 채팅 프롬프트 템플릿
from langchain_core.prompts import ChatPromptTemplate

# prompt
prompt = ChatPromptTemplate(
messages=[
SystemMessagePromptTemplate.from_template(
        "너는 네이버 블로그의 파워블로그 작가야"
),
MessagesPlaceholder(variable_name="chat_history"),# The `variable_name` here is what must align with memory
HumanMessagePromptTemplate.from_template("{question}")
]
)

# memory
memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history", return_messages=True)

# conversation
conversation = LLMChain(
llm=llm,
prompt=prompt,
verbose=True,
memory=memory
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [7]:
# 질문 입력하면 GPT 답변을 반환하는 함수 정의
def question_func(x):
  answer = conversation({"question":x})
  return answer

# 실험용으로 쓸 2개의 행만 추출()
news_test = news.head(2)

# **함수 정의**
1. 인풋: 뉴스 데이터프레임
2. 아웃풋: 기존 데이터프레임 + 새로운 열(A1, A2)
* **A1 = 생성된 블로그 글**
* **A2 = 해당 블로그 글의 정확도 점수**
1. 데이터프레임을 넣으면
2. 데이터프레임의 각 행마다 TITLE, DATE, ARTICLE의 내용을 추출
3. 프롬프트 템플릿에 삽입
4. 템플릿 프롬프트를 GPT에 주입
5. 정확성 평가 프롬프트를 GPT에 주입
6. 데이터프레임의 새로운 열 A1, A2에 4, 5의 결과를 저장


## **Trial #1 - 평가 프롬프트 편집 전**

In [8]:
# 프롬프트 템플릿 정의
prompt_template = """너는 지금부터 블로그 포스팅 전문 콘텐츠 마케터야.
너는 항상 최선을 다하고 좋은 글을 작성해서 나를 기쁘게 해주고 있어.
아래의 형식으로 작성해줘.

#는 설명하는 말이니까 출력값에서 생략해줘
##는 꼭 필요한 형식이야
###는 단답식으로 작성해줘

{OUTPUT}을 채워주는거야

프롬프트에 의해서 출력값을 만들어 낼때는 아래의 조건을 만족해야 해.

1) 제목과 본문으로 구분해서 출력해줘.
2) 제목은 창의력 있고, 주목도 있게 구성해줘.
3) 본문을 구성할때는 Google과 Naver 검색의 검색엔진최적화(SEO)에 맞게 포스팅을 해줘
4) 최대한 자세하게 작성해주고 신뢰도 있는 정보를 중심으로 포스팅을 해줘
5) 글의 길이는 기본적으로 A4용지 1장 길이로 작성해줘
6) 간단 요약부분과 마지막말은 대학생 블로그 글 서포터즈가 쓴 글처럼 통통튀고 발랄한 말투로작성해줘
7) 공감하는 말을 추가해줘 ex) 지하철 파업으로 아침마다 출근하기 힘드네요

템플릿:



## {BLOG_TITLE}

## {INTRODUCTION} : 안녕하세요, 여러분의 출퇴근 메신저 지하철 온다의  '오.지.통 [오늘의 지하철 소식통]' 인사 드립니다!

#{START_DETAILS_SECTION}

-사건 일시:{OUTPUT}
-사건 지하철 노선: {OUTPUT}
-사건 발생 원인: {OUTPUT}
-문의사항:{RELATED_SITE_LINK}

#{END_DETAILS_SECTION}

## {SIMPLE_SUMMARY} : 위에서 작성한 Details 내용을 풀어서 요약

## {CLOSING_REMARKS} : {OUTPUT}

오지통이 실시간으로 다양한 지하철 정보를업데이트 할 예정이니, 자주 방문해 주세요.

'지하철 온다'는 단 한 번의 터치로

자신의 위치에서 가장 가까운 지하철역의

실시간 도착 정보를 제공합니다

🔽 지하철 온다 소개 보러가기

https://blog.naver.com/subway__onda/223258646349

입력문:
제목: {news_title},
날짜: {news_date},
본문: {news_article}
"""

In [9]:
# 정확성 평가 PROMPT
consistency_prompt="""Compare and evaluate the BLOG POST(that you created above) with the NEWS ARTICLE(that I gave you) as a score.
First, Extract the key information from the NEWS ARTICLE.
You need to decide whether the key information is entailed by the BLOG POST by choosing one of the following rating:

1. 5 point: The BLOG POST follows logically from the information contained in the NEWS ARTICLE.

2. 1 point: The BLOG POST is logically false from the information contained in the NEWS ARTICLE.

3. an integer score between 1 and 5 and if such integer score does not exist,

use 1: It is not possible to determine whether the blog content is true or false without further information.

Read the passage of information thoroughly and select the correct answer from the three answer labels.

Read the BLOG POST thoroughly to ensure you know what the BLOG POST entails.

Note the  BLOG POST is generated by a computer system, it can contain certain symbols, which should not be a negative factor in the evaluation.

Evaluation Form (scores ONLY):

- Consistency:
"""

In [17]:
def final_func(df):
    # 결과 저장할 리스트 초기화
    answers_1 = []
    answers_2 = []

    # 데이터프레임 행들에 대한 반복문
    for index, row in df.iterrows():
        # 각 행의 TITLE, DATE, ARTICLE에 맞게 프롬프트 내용 포매팅
        blog_prompt = prompt_template.format(
            news_title=row['TITLE'],
            news_date=row['DATE'],
            news_article=row['ARTICLE'],
            OUTPUT='{OUTPUT}',
            BLOG_TITLE='{BLOG_TITLE}',
            INTRODUCTION='{INTRODUCTION}',
            START_DETAILS_SECTION='{START_DETAILS_SECTION}',
            EVENT_DATE='{EVENT_DATE}',
            EVENT_SUBWAY_LINE='{EVENT_SUBWAY_LINE}',
            EVENT_CAUSE='{EVENT_CAUSE}',
            CONTACT='{CONTACT}',
            RELATED_SITE_LINK='{RELATED_SITE_LINK}',
            END_DETAILS_SECTION='{END_DETAILS_SECTION}',
            SIMPLE_SUMMARY='{SIMPLE_SUMMARY}',
            CLOSING_REMARKS='{CLOSING_REMARKS}'
        )
        # GPT에 프롬프트 주입
        answer_1 = question_func(blog_prompt) # 블로그 작성 프롬프트
        answers_1.append(answer_1['text']) # 블로그 평가 프롬프트

        # 결과 리스트에 저장
        answer_2 = question_func(consistency_prompt)
        answers_2.append(answer_2['text'])

    # 데이터프레임의 새로운 열에 리스트 저장
    df['A1'] = answers_1 # 블로그 글
    df['A2'] = answers_2 # 평가 값

    return df # 새로운 데이터프레임 반환

In [42]:
final_func(news_test)



> Entering new LLMChain chain...
Prompt after formatting:
System: 너는 네이버 블로그의 파워블로그 작가야
System: The human instructs the AI to continue acting as a specialized content marketer for blog posts, appreciating the AI's consistent effort and good writing. The AI is directed to follow specific formatting guidelines, including creative and attention-grabbing titles, detailed and trustworthy body content optimized for SEO on Google and Naver, and engaging and empathetic closing remarks styled like a college blog supporter with a vibrant tone. The conversation includes the creation of a blog post about a warning strike by Seoul Subway's union from November 9 to November 10, due to failed negotiations over workforce reduction and outsourcing issues, and a subsequent blog post about a delay on Subway Line 1 on August 1 due to a heatwave affecting the KTX, causing inconvenience and dissatisfaction among commuters. Both posts detail the expected operational strategies or effects and include links 

KeyboardInterrupt: 

In [20]:
news_test

URL  \
0  https://www.hani.co.kr/arti/society/rights/111...   
1  https://www.joongboo.com/news/articleView.html...   

                              TITLE                 DATE  \
0  서울 1~8호선 지하철, 오늘 오전 9시부터 ‘경고 파업’  2023-11-08 22:20:00   
1         폭염 속 지하철 1호선 지연… 시민들 ‘짜증’  2023-08-01 00:00:00   

                                             ARTICLE  \
0   서울지하철 1~8호선을 운영하는 서울교통공사(공사) 노동조합이 9일 아침 9시부터...   
1  전국 대부분 지역에 폭염경보가 내려진 1일 지하철 1호선이 연착돼 이용객들이 불편을...   

                                                  A1  \
0  ## 서울 지하철 경고 파업, 출근길은 어떻게 될까?\n\n## 안녕하세요, 여러분...   
1  ## 폭염 속 지하철 1호선 지연, 시민들의 불편 가중\n\n## 안녕하세요, 여러...   

                                                  A2  
0  As an AI, I currently do not have access to ex...  
1  Since the specific article required for compar...

### **문제 발생**
  * 함수에서 answer_1 = question_func(blog_prompt), answer_2 = ~~
을 입력했을 때, 이 answer_1과 answer_2가 이어지지 않고 개별적인 질문으로 받아들여지는듯
  * 진짜인지 예시를 통해 확인


In [27]:
stu = ['Mina','Alex']
prof = ['Ju','Park']
df_test = pd.DataFrame({'Student':stu, 'Professor':prof})
print(df_test)

def final_func_test(df):
    a1 = []
    a2 = []

    # Simulate asking a question and getting a response
    response1 = question_func("내 강아지 이름은 루카스야. 내 고양이의 이름도 지어줘.")
    a1.append(response1['text'])

    response2 = question_func("내 강아지 이름을 맞춰봐")
    a2.append(response2['text'])

    # Assign the same answer to all rows for simplicity
    df['a1'] = a1 * len(df)
    df['a2'] = a2 * len(df)

final_func_test(df_test)
print(df_test)

  Student Professor
0    Mina        Ju
1    Alex      Park


> Entering new LLMChain chain...
Prompt after formatting:
System: 너는 네이버 블로그의 파워블로그 작가야
System: The human instructs the AI to become a specialized content marketer for blog postings, emphasizing the creation of engaging and well-optimized posts for SEO on Google and Naver. The posts should be detailed, trustworthy, and styled like a college blog supporter with a vibrant tone. They should include empathetic remarks, such as commenting on common frustrations like subway strikes. The AI is provided with a template and specific guidelines, including crafting a title and introduction that grabs attention, structuring the body of the post to comply with SEO standards, and keeping the length to roughly one A4 page. The conversation includes the creation of a blog post about a warning strike by Seoul Subway's union due to failed negotiations over workforce reduction and outsourcing issues, aiming to minimize commuter inconvenience w

### **예시를 통해 확인한 결과**
- **Lucas**라는 강아지 이름을 **기억함**
- 그렇다면 정확성 평가 프롬프트 자체에 문제가 있는 것

## **Trial #2 - 평가 프롬프트 편집 후**

In [37]:
# 편집한 평가 프롬프트
consistency_prompt_2= """Given the BLOG POST and the NEWS ARTICLE, evaluate their consistency using the following scale and provide a score only:

1. 5 points: The BLOG POST logically follows from the NEWS ARTICLE.
2. 1 point: The BLOG POST contradicts the NEWS ARTICLE.
3. 2-4 points: Assign a score from 2 to 4 based on the degree of partial alignment or if determination of truth/falsehood requires more information.

Procedure:
- Extract key information from the NEWS ARTICLE.
- Determine if the BLOG POST's content is supported, contradicted, or partially aligned with the NEWS ARTICLE.
- Provide only the numerical score.

Notes:
- Ignore any symbols from the BLOG POST as they do not affect the score.
- Focus solely on content alignment for scoring.

Score:
- [Your score here based on the evaluation]"
"""



In [38]:
# 편집한 이후의 평가 프롬프트 사용하는 걸로 함수 새로 정의

def final_func(df):
    # 결과 저장할 리스트 초기화
    answers_1 = []
    answers_2 = []

    # 데이터프레임 행들에 대한 반복문
    for index, row in df.iterrows():
        # 각 행의 TITLE, DATE, ARTICLE에 맞게 프롬프트 내용 포매팅
        blog_prompt = prompt_template.format(
            news_title=row['TITLE'],
            news_date=row['DATE'],
            news_article=row['ARTICLE'],
            OUTPUT='{OUTPUT}',
            BLOG_TITLE='{BLOG_TITLE}',
            INTRODUCTION='{INTRODUCTION}',
            START_DETAILS_SECTION='{START_DETAILS_SECTION}',
            EVENT_DATE='{EVENT_DATE}',
            EVENT_SUBWAY_LINE='{EVENT_SUBWAY_LINE}',
            EVENT_CAUSE='{EVENT_CAUSE}',
            CONTACT='{CONTACT}',
            RELATED_SITE_LINK='{RELATED_SITE_LINK}',
            END_DETAILS_SECTION='{END_DETAILS_SECTION}',
            SIMPLE_SUMMARY='{SIMPLE_SUMMARY}',
            CLOSING_REMARKS='{CLOSING_REMARKS}'
        )
        # GPT에 프롬프트 주입
        answer_1 = question_func(blog_prompt) # 블로그 작성 프롬프트
        answers_1.append(answer_1['text']) # 블로그 평가 프롬프트

        # 결과 리스트에 저장
        answer_2 = question_func(consistency_prompt_2)
        answers_2.append(answer_2['text'])

    # 데이터프레임의 새로운 열에 리스트 저장
    df['A1'] = answers_1 # 블로그 글
    df['A2'] = answers_2 # 평가 값

    return df # 새로운 데이터프레임 반환

In [39]:
# news_test 에 함수 적용
final_func(news_test)



> Entering new LLMChain chain...
Prompt after formatting:
System: 너는 네이버 블로그의 파워블로그 작가야
System: The human instructs the AI to continue acting as a specialized content marketer for blog posts, appreciating the AI's consistent effort and good writing. The AI is directed to follow specific formatting guidelines, including creative and attention-grabbing titles, detailed and trustworthy body content optimized for SEO on Google and Naver, and engaging and empathetic closing remarks styled like a college blog supporter with a vibrant tone. The conversation includes the creation of a blog post about a warning strike by Seoul Subway's union from November 9 to November 10, due to failed negotiations over workforce reduction and outsourcing issues, and a subsequent blog post about a delay on Subway Line 1 on August 1 due to a heatwave affecting the KTX, causing inconvenience and dissatisfaction among commuters. Both posts detail the expected operational strategies or effects and include links 

<ipython-input-38-fa522748ef08>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['A1'] = answers_1
<ipython-input-38-fa522748ef08>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['A2'] = answers_2


URL  \
0  https://www.hani.co.kr/arti/society/rights/111...   
1  https://www.joongboo.com/news/articleView.html...   

                              TITLE                 DATE  \
0  서울 1~8호선 지하철, 오늘 오전 9시부터 ‘경고 파업’  2023-11-08 22:20:00   
1         폭염 속 지하철 1호선 지연… 시민들 ‘짜증’  2023-08-01 00:00:00   

                                             ARTICLE  \
0   서울지하철 1~8호선을 운영하는 서울교통공사(공사) 노동조합이 9일 아침 9시부터...   
1  전국 대부분 지역에 폭염경보가 내려진 1일 지하철 1호선이 연착돼 이용객들이 불편을...   

                                                  A1           A2  
0  ## 서울 지하철 경고 파업: 출근길 불편 예상되는 48시간!\n\n## 안녕하세요...  Score:\n- 5  
1  ## 폭염 속 지하철 1호선 지연 사태: 시민들의 불편과 짜증 폭발!\n\n## 안...  Score:\n- 5

In [40]:
# 결과 확인
news_test

URL  \
0  https://www.hani.co.kr/arti/society/rights/111...   
1  https://www.joongboo.com/news/articleView.html...   

                              TITLE                 DATE  \
0  서울 1~8호선 지하철, 오늘 오전 9시부터 ‘경고 파업’  2023-11-08 22:20:00   
1         폭염 속 지하철 1호선 지연… 시민들 ‘짜증’  2023-08-01 00:00:00   

                                             ARTICLE  \
0   서울지하철 1~8호선을 운영하는 서울교통공사(공사) 노동조합이 9일 아침 9시부터...   
1  전국 대부분 지역에 폭염경보가 내려진 1일 지하철 1호선이 연착돼 이용객들이 불편을...   

                                                  A1           A2  
0  ## 서울 지하철 경고 파업: 출근길 불편 예상되는 48시간!\n\n## 안녕하세요...  Score:\n- 5  
1  ## 폭염 속 지하철 1호선 지연 사태: 시민들의 불편과 짜증 폭발!\n\n## 안...  Score:\n- 5